In [1]:
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.linear_model import LogisticRegression
import re
from sklearn.decomposition import LatentDirichletAllocation
import gensim
from gensim import corpora

c:\python\python37\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [3]:
stopwords = stopwords.words('english')[:]

In [4]:
pd.set_option("display.max_rows",270)

# Task A

### Engagement Score

In [5]:
post = pd.read_csv("insta_data_with_labels.csv").iloc[:,1:]

In [6]:
post.shape

(270, 6)

In [7]:
max_like = max(post["likes"])
max_like

1539774

In [8]:
max_like = max(post["likes"])
list_norm_like = []
for p in post.itertuples():
    norm_like = p.likes/max_like
    list_norm_like.append(norm_like)
post["norm_like"] = list_norm_like

max_comment = max(post["comments"])
list_norm_comment = []
for p in post.itertuples():
    norm_comment = p.comments/max_comment
    list_norm_comment.append(norm_comment)
    
post["norm_comment"]= list_norm_comment

In [9]:
engagement_score = []
for p in post.itertuples():
    e_score = .4*p.norm_like+.6*p.norm_comment
    engagement_score.append(e_score)
engagement_score

[0.017560655627206684,
 0.02366236449064101,
 0.037809338825375745,
 0.09019406690899491,
 0.11252898280760024,
 0.12070288926685988,
 0.0684502432182017,
 0.07439259702505385,
 0.029396026017428026,
 0.044150234902194656,
 0.08526724827039955,
 0.06025193407719019,
 0.12698927056147763,
 0.15885322135378185,
 0.06036176590398917,
 0.062475533746834275,
 0.07939817644979734,
 0.10823153463623596,
 0.06637011537807715,
 0.0524855773452004,
 0.09030157519960061,
 0.08801654248682615,
 0.07918353096459323,
 0.10909306582838439,
 0.33221843192878076,
 0.08145705246775684,
 0.04558561823412439,
 0.12271770638568008,
 0.058085794017089146,
 0.0878135163021513,
 0.032037474945001886,
 0.03397843335917866,
 0.03537797494481368,
 0.05954897098807081,
 0.26121185644432376,
 0.24268638545740612,
 0.14122382089357513,
 0.1482207042829119,
 0.09477811178034269,
 0.043401602872195966,
 0.1697252296637716,
 0.05805284878448901,
 0.02543701615277202,
 0.040006916819220216,
 0.05253045494082848,
 0.097

In [10]:
med_e_score = np.median(engagement_score)

In [11]:
med_e_score

0.0878759068571878

In [12]:
top_quartile = np.percentile(engagement_score, 75)

In [13]:
top_quartile

0.12748719610916837

In [14]:
bot_quartile = np.percentile(engagement_score, 25)

In [15]:
bot_quartile

0.05885066955920289

In [16]:
post["engagement_score"] = engagement_score

In [17]:
eng_score = []
for i in range(len(engagement_score)):
    if engagement_score[i]>=med_e_score:
        e_score = 1
    else:
        e_score = 0
    eng_score.append(e_score)
eng_score
post["engagement"] = eng_score

## Part C

In [143]:
# sanity check
post.head()

,Unnamed: 0.1,urls,captions,comments,likes,labels,norm_like,norm_comment,engagement_score,engagement
0,0,https://scontent-sea1-1.cdninstagram.com/vp/7831adc9c2d624937ec8197b4affcded/5C76DA0B/t51.2885-15/e15/c181.0.718.718/s480x480/44904753_194498844802328_8238377254064471543_n.jpg?ig_cache_key=MTkxMjcyNTE4Njk1Njg2MzM4Mg%3D%3D.2.c,"Photo by @amivitale | Ntipiyon Nonguta and her son Bernard, one, relax with their neighbors as they make beaded belts for theLoisaba Community Trust in Ewaso Village in Laikipia, Kenya. These women are in a community surrounding Loisaba Conservancy (@loisaba_conservancy) and are being supported and given access to international markets. Revenue from ecotourism is reinvested into neighboring communities like #Ewaso . To get involved and learn more about community based conservation, please follow @loisaba_conservancy and @amivitale. @nature_org @natgeoimagecollection #kenya #magicalkenya #whyilovekenya #northernkenya #africa #children #mothersandsons #mothers #empowerment #conservation #everydayafrica #photojournalism #amivitale",278,61459,"['people', 'public space', 'community', 'tribe', 'tree', 'travel', 'temple', 'recreation', 'tourism', 'crowd']",0.039914,0.002658,0.017561,0
1,1,https://scontent-sea1-1.cdninstagram.com/vp/53f7539662906933bbbce050cd4fdc2c/5BEF2CD7/t51.2885-15/e15/s640x640/45543088_970491189813847_3800824331312248168_n.jpg,"Video by @amivitale | Wild adult elephants drink and cool off at a watering on the Namunyak Wildlife Conservancy. An elephant’s trunk typically holds around half of a gallon of water, but a big bull elephant can hold as much as 2.5 gallons. After spraying themselves, the elephants will often coat themselves with dust which acts as a bug repellent and sunblock. Namunyak Wildlife Conservancy is home to Reteti Elephant Sanctuary (@r.e.s.c.u.e), the first ever community-owned and run elephant sanctuary in Africa. Reteti operates in partnership with Conservation International (@conservationorg) who provide critical operational support and work to scale the Reteti community-centered model to create lasting impacts worldwide. Follow @amivitale @conservationorg & @r.e.s.c.u.e to learn about the incredible work being done to protect and preserve these magical beings. @nrt_kenya @kenyawildlifeservice @sandiegozoo @tusk_org @thephotosociety @natgeo #namunyak #protectelephants #bekindtoelephants #DontLetThemDisappear #elephants #saveelephants #stoppoaching #kenya #northernkenya #magicalkenya #whyilovekenya #africa #everydayafrica #natureisspeaking #photojournalism #amivitale",362,83092,NaN,0.053964,0.003461,0.023662,0
2,2,https://scontent-sea1-1.cdninstagram.com/vp/1e4ff18aa09db5069b6e1c6e9e65eac0/5C6F3508/t51.2885-15/e15/s480x480/44577672_336304220507048_1738524933735107944_n.jpg?ig_cache_key=MTkxMjQyNTAwNDk3MzMzNTE0NQ%3D%3D.2,"Photo by George Steinmetz @geosteinmetz | Epic flying last week over the Pyramids of Giza. I had tried for years, and was turned down even with Egyptian military helicopters. Last week Egypt hosted the world paramotor championships, and suddenly the impossible happened. This is me, the last one to land at sunset. For a rare view of the pyramids from above, go to @geosteinmetz",431,136026,"['air sports', 'paragliding', 'landscape', 'sand', 'ecoregion', 'desert', 'powered paragliding', 'windsports', 'sahara', 'steppe']",0.088342,0.004121,0.037809,0
3,3,https://scontent-sea1-1.cdninstagram.com/vp/a43b38e630c9b7b76c93d76517e4882e/5C8A9361/t51.2885-15/e15/s480x480/42697363_1856252364473290_1258265168604898264_n.jpg?ig_cache_key=MTkxMjM1MDIwNDMwMDk5NzU5Mg%3D%3D.2,"Photo by @irablockphoto | A nighttime view from the top of the Old Town Bridge Tower of the Charles Bridge and Prague Castle along the banks of the Vltava River in Prague, Czech Republic. The Charles Bridge was constructed under King Charles IV of Bohemia starting in 1357 and was finished in the begging of the 15th century. The bridge is a very popular tourist destination especially at sunset and

In [144]:
post.shape

(270, 10)

In [145]:
# use relevant columns - labels and engagement_score
post_clean = post.drop(post.columns[[0,1,2,3,4,6,7,9]], axis = 1)

In [21]:
post_clean.head()

,labels,engagement_score
0,"['people', 'public space', 'community', 'tribe', 'tree', 'travel', 'temple', 'recreation', 'tourism', 'crowd']",0.017561
1,NaN,0.023662
2,"['air sports', 'paragliding', 'landscape', 'sand', 'ecoregion', 'desert', 'powered paragliding', 'windsports', 'sahara', 'steppe']",0.037809
3,"['waterway', 'city', 'cityscape', 'reflection', 'sky', 'landmark', 'river', 'town', 'evening', 'dusk']",0.090194
4,"['mountain range', 'mountainous landforms', 'massif', 'nunatak', 'geological phenomenon', 'arête', 'piste', 'aerial photography', 'glacial landform', 'ridge']",0.112529


In [146]:
post_clean = post_clean.dropna()

In [147]:
post_clean.shape

(238, 2)

In [148]:
post_clean

,labels,engagement_score
0,"['people', 'public space', 'community', 'tribe', 'tree', 'travel', 'temple', 'recreation', 'tourism', 'crowd']",0.017561
2,"['air sports', 'paragliding', 'landscape', 'sand', 'ecoregion', 'desert', 'powered paragliding', 'windsports', 'sahara', 'steppe']",0.037809
3,"['waterway', 'city', 'cityscape', 'reflection', 'sky', 'landmark', 'river', 'town', 'evening', 'dusk']",0.090194
4,"['mountain range', 'mountainous landforms', 'massif', 'nunatak', 'geological phenomenon', 'arête', 'piste', 'aerial photography', 'glacial landform', 'ridge']",0.112529
5,"['marine mammal', 'whales dolphins and porpoises', 'mammal', 'water', 'marine biology', 'killer whale', 'dolphin', 'short beaked common dolphin', 'stenella', 'organism']",0.120703
6,"['highland', 'mountain', 'mountainous landforms', 'ridge', 'badlands', 'sky', 'wilderness', 'fell', 'escarpment', 'national park']",0.068450
7,"['black and white', 'infrastructure', 'crowd', 'monochrome photography', 'street', 'monochrome', 'temple', 'religion']",0.074393
9,"['plant', 'tree', 'recreation', 'city', 'leisure', 'park', 'vehicle', 'sky', 'grass', 'tourism']",0.044150
10,"['ecosystem', 'tree', 'vegetation', 'wildlife', 'nature reserve', 'fauna', 'wilderness', 'shrubland', 'woodland', 'grass']",0.085267
11,"['sky', 'horizon', 'sea', 'calm', 'morning', 'dawn', 'loch', 'sunrise', 'atmosphere', 'highland']",0.060252


In [149]:
# clean up
def clean_up(label):
    label = label.replace("[", "")
    label = label.replace("]", "")
    label = label.replace("'", "")
    label = label.replace(",", " ")
    return label

In [150]:
# convert to series and clean up the labels column
post_clean["labels"] = post_clean.iloc[:,0].apply(clean_up)

In [151]:
post_clean

,labels,engagement_score
0,people public space community tribe tree travel temple recreation tourism crowd,0.017561
2,air sports paragliding landscape sand ecoregion desert powered paragliding windsports sahara steppe,0.037809
3,waterway city cityscape reflection sky landmark river town evening dusk,0.090194
4,mountain range mountainous landforms massif nunatak geological phenomenon arête piste aerial photography glacial landform ridge,0.112529
5,marine mammal whales dolphins and porpoises mammal water marine biology killer whale dolphin short beaked common dolphin stenella organism,0.120703
6,highland mountain mountainous landforms ridge badlands sky wilderness fell escarpment national park,0.068450
7,black and white infrastructure crowd monochrome photography street monochrome temple religion,0.074393
9,plant tree recreation city leisure park vehicle sky grass tourism,0.044150
10,ecosystem tree vegetation wildlife nature reserve fauna wilderness shrubland woodland grass,0.085267
11,sky horizon sea calm morning dawn loch sunrise atmosphere highland,0.060252


In [152]:
# throw these string of labels into a list
final_labels = post_clean["labels"].tolist()

In [153]:
final_labels

['people  public space  community  tribe  tree  travel  temple  recreation  tourism  crowd',
 'air sports  paragliding  landscape  sand  ecoregion  desert  powered paragliding  windsports  sahara  steppe',
 'waterway  city  cityscape  reflection  sky  landmark  river  town  evening  dusk',
 'mountain range  mountainous landforms  massif  nunatak  geological phenomenon  arête  piste  aerial photography  glacial landform  ridge',
 'marine mammal  whales dolphins and porpoises  mammal  water  marine biology  killer whale  dolphin  short beaked common dolphin  stenella  organism',
 'highland  mountain  mountainous landforms  ridge  badlands  sky  wilderness  fell  escarpment  national park',
 'black and white  infrastructure  crowd  monochrome photography  street  monochrome  temple  religion',
 'plant  tree  recreation  city  leisure  park  vehicle  sky  grass  tourism',
 'ecosystem  tree  vegetation  wildlife  nature reserve  fauna  wilderness  shrubland  woodland  grass',
 'sky  horizon

In [154]:
# init count vectorizer
vectorizer = CountVectorizer(analyzer='word',       
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                            )

In [155]:
data_vectorized = vectorizer.fit_transform(final_labels)

## LDA model with 4 topics

In [156]:
lda = LatentDirichletAllocation(n_components = 6, random_state = 0)

In [157]:
lda.fit(data_vectorized)

c:\python\python37\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=6, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [158]:
lda.n_components

6

## Which Words Load on Which Topics (Weights of Each Keyword in Each Topic)

In [159]:
# column names
topicnames = ["Topic" + str(i) for i in range(1, lda.n_components + 1)]

# weights of each keyword in each topic

# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda.components_.T)

# Assign Column and Index
df_topic_keywords.index = vectorizer.get_feature_names()
df_topic_keywords.columns = topicnames

# View
df_topic_keywords

,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6
abdomen,0.232944,0.215697,0.226181,0.219692,1.281541,0.213215
accessory,1.054471,2.111974,1.283405,0.217115,0.221841,0.207704
accipitriformes,0.233699,0.221102,0.228149,1.062371,0.223267,0.228506
adventure,0.214649,1.692356,0.213931,0.334155,0.213432,1.790808
advertising,0.220114,0.228623,0.219820,1.040164,0.224346,0.220003
aeolian,0.223491,0.225654,2.037220,0.251282,0.218932,0.264056
aerial,0.452700,0.279628,1.266694,3.484535,0.215822,0.207506
african,5.876329,0.213844,0.214889,0.226692,0.211241,0.217884
agriculture,0.223709,2.103468,0.218017,0.233473,0.215909,0.223850
air,0.222980,0.217158,2.115748,0.209445,0.206544,0.231310


In [160]:
# throw this onto a csv
df_topic_keywords.to_csv("wordstotopics2.csv")

In [177]:
# Topic 1 
topic1 = pd.DataFrame(df_topic_keywords.sort_values(by = ['Topic1'], ascending = False).head().index.values)
topic1.columns = ["Topic1"]
topic1

,Topic1
0,elephant
1,photography
2,animal
3,mammoths
4,elephants


We decided to name this topic "Big Mammals"

In [178]:
# topic 2
topic2 = pd.DataFrame(df_topic_keywords.sort_values(by = ['Topic2'], ascending = False).head().index.values)
topic2.columns = ["Topic2"]
topic2

,Topic2
0,tree
1,sky
2,recreation
3,water
4,mountain


We decided to name this topic "Nature"

In [180]:
# topic 3
topic3 = pd.DataFrame(df_topic_keywords.sort_values(by = ['Topic3'], ascending = False).head().index.values)
topic3.columns = ["Topic3"]
topic3

,Topic3
0,sky
1,ecosystem
2,grassland
3,area
4,city


We decided to name this topic "ecosystem"

In [181]:
# topic 4
topic4 = pd.DataFrame(df_topic_keywords.sort_values(by = ['Topic4'], ascending = False).head().index.values)
topic4.columns = ["Topic4"]
topic4

,Topic4
0,wildlife
1,mammal
2,fauna
3,organism
4,animal


We decided to name this topic "Wildlife"

In [183]:
# topic 5
topic5 = pd.DataFrame(df_topic_keywords.sort_values(by = ['Topic5'], ascending = False).head().index.values)
topic5.columns = ["Topic5"]
topic5

,Topic5
0,human
1,girl
2,temple
3,horse
4,site


We decided to name this topic "People & Exploration"

### Topic Weights For Each Image

In [184]:
# Create Document - Topic Matrix
lda_output = lda.transform(data_vectorized)

# index names
postnames = ["Post" + str(i) for i in range(1, len(final_labels) + 1)]

# Make the pandas dataframe
df_post_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=postnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_post_topic.values, axis=1)
for i in range(len(dominant_topic)):
    dominant_topic[i] = dominant_topic[i] + 1
    
# df_post_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_post_topics = df_post_topic.style.applymap(color_green).applymap(make_bold)
df_post_topics

,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6
Post1,0.01,0.93,0.01,0.01,0.01,0.01
Post2,0.01,0.01,0.94,0.01,0.01,0.01
Post3,0.02,0.02,0.92,0.02,0.02,0.02
Post4,0.01,0.43,0.01,0.53,0.01,0.01
Post5,0.01,0.01,0.01,0.96,0.01,0.01
Post6,0.01,0.94,0.01,0.01,0.01,0.01
Post7,0.92,0.02,0.02,0.02,0.02,0.02
Post8,0.02,0.92,0.02,0.02,0.02,0.02
Post9,0.01,0.01,0.01,0.93,0.01,0.01
Post10,0.02,0.02,0.92,0.02,0.02,0.02


In [186]:
df_post_topic = df_post_topic.rename(columns={'Topic1': 'Big Mammals', 'Topic2': 'Nature', 'Topic3': 'Ecosystem', 'Topic4':'Wildlife', 'Topic5': 'People & Exploration', 'Topic6':'Terrain'})

In [191]:
df_post_topic

,Big Mammals,Nature,Ecosystem,Wildlife,People & Exploration,Terrain
Post1,0.01,0.93,0.01,0.01,0.01,0.01
Post2,0.01,0.01,0.94,0.01,0.01,0.01
Post3,0.02,0.02,0.92,0.02,0.02,0.02
Post4,0.01,0.43,0.01,0.53,0.01,0.01
Post5,0.01,0.01,0.01,0.96,0.01,0.01
Post6,0.01,0.94,0.01,0.01,0.01,0.01
Post7,0.92,0.02,0.02,0.02,0.02,0.02
Post8,0.02,0.92,0.02,0.02,0.02,0.02
Post9,0.01,0.01,0.01,0.93,0.01,0.01
Post10,0.02,0.02,0.92,0.02,0.02,0.02


In [192]:
post_clean

,labels,engagement_score
0,people public space community tribe tree travel temple recreation tourism crowd,0.017561
2,air sports paragliding landscape sand ecoregion desert powered paragliding windsports sahara steppe,0.037809
3,waterway city cityscape reflection sky landmark river town evening dusk,0.090194
4,mountain range mountainous landforms massif nunatak geological phenomenon arête piste aerial photography glacial landform ridge,0.112529
5,marine mammal whales dolphins and porpoises mammal water marine biology killer whale dolphin short beaked common dolphin stenella organism,0.120703
6,highland mountain mountainous landforms ridge badlands sky wilderness fell escarpment national park,0.068450
7,black and white infrastructure crowd monochrome photography street monochrome temple religion,0.074393
9,plant tree recreation city leisure park vehicle sky grass tourism,0.044150
10,ecosystem tree vegetation wildlife nature reserve fauna wilderness shrubland woodland grass,0.085267
11,sky horizon sea calm morning dawn loch sunrise atmosphere highland,0.060252


In [202]:
post_clean.tail()

,labels,engagement_score
265,sport venue man fan stadium structure player jersey product outerwear competition event,0.062915
266,reflection water waterway night tree city evening street screenshot sky,0.106362
267,fauna wildlife nature reserve primate jungle terrestrial animal organism forest rainforest grass,0.085054
268,aviation aircraft air travel sky,0.042053
269,mountain mountainous landforms sky ridge geological phenomenon adventure atmosphere cloud terrain mountaineer,0.104143


In [209]:
df_post_topic = df_post_topic.reset_index()

In [208]:
post_clean = post_clean.reset_index()

In [210]:
df_post_topic["engagement"] = post_clean["engagement_score"]

In [211]:
df_post_topic

,index,Big Mammals,Nature,Ecosystem,Wildlife,People & Exploration,Terrain,engagement
0,Post1,0.01,0.93,0.01,0.01,0.01,0.01,0.017561
1,Post2,0.01,0.01,0.94,0.01,0.01,0.01,0.037809
2,Post3,0.02,0.02,0.92,0.02,0.02,0.02,0.090194
3,Post4,0.01,0.43,0.01,0.53,0.01,0.01,0.112529
4,Post5,0.01,0.01,0.01,0.96,0.01,0.01,0.120703
5,Post6,0.01,0.94,0.01,0.01,0.01,0.01,0.068450
6,Post7,0.92,0.02,0.02,0.02,0.02,0.02,0.074393
7,Post8,0.02,0.92,0.02,0.02,0.02,0.02,0.044150
8,Post9,0.01,0.01,0.01,0.93,0.01,0.01,0.085267
9,Post10,0.02,0.02,0.92,0.02,0.02,0.02,0.060252


In [212]:
top_quartile

0.12748719610916837

In [213]:
bot_quartile

0.05885066955920289

## Top Quartile 

In [216]:
# Top Quartile of engagement 
top_q = df_post_topic[df_post_topic.apply(lambda x: x['engagement'] > top_quartile, axis=1)]
top_q.head()

,index,Big Mammals,Nature,Ecosystem,Wildlife,People & Exploration,Terrain,engagement
11,Post12,0.01,0.01,0.01,0.93,0.01,0.01,0.158853
21,Post22,0.03,0.03,0.03,0.83,0.03,0.03,0.332218
31,Post32,0.01,0.01,0.01,0.94,0.01,0.01,0.261212
32,Post33,0.01,0.01,0.94,0.01,0.01,0.01,0.242686
33,Post34,0.01,0.01,0.01,0.01,0.94,0.01,0.141224


In [232]:
top_q = []

In [233]:
top_q_list.append(top_q["Big Mammals"].mean())

In [234]:
top_q_list.append(top_q["Nature"].mean())

In [235]:
top_q_list.append(top_q["Ecosystem"].mean())

In [236]:
top_q_list.append(top_q["Wildlife"].mean())

In [237]:
top_q_list.append(top_q["People & Exploration"].mean())

In [238]:
top_q_list.append(top_q["Terrain"].mean())

## Bot Quartile

In [223]:
# Bot Quartile of engagement 
bot_q = df_post_topic[df_post_topic.apply(lambda x: x['engagement'] < bot_quartile, axis=1)]
bot_q.head()

,index,Big Mammals,Nature,Ecosystem,Wildlife,People & Exploration,Terrain,engagement
0,Post1,0.01,0.93,0.01,0.01,0.01,0.01,0.017561
1,Post2,0.01,0.01,0.94,0.01,0.01,0.01,0.037809
7,Post8,0.02,0.92,0.02,0.02,0.02,0.02,0.044150
16,Post17,0.03,0.86,0.03,0.03,0.03,0.03,0.052486
23,Post24,0.01,0.17,0.78,0.01,0.01,0.01,0.045586


In [239]:
bot_q_list = []

In [240]:
bot_q_list.append(bot_q["Big Mammals"].mean())

In [241]:
bot_q_list.append(bot_q["Nature"].mean())

In [242]:
bot_q_list.append(bot_q["Ecosystem"].mean())

In [243]:
bot_q_list.append(bot_q["Wildlife"].mean())

In [244]:
bot_q_list.append(bot_q["People & Exploration"].mean())

In [245]:
bot_q_list.append(bot_q["Terrain"].mean())

In [254]:
table = ["Big Mammals", "Nature", "Ecosystem", "Wildlife", "People & Exploration", "Terrain"]

In [255]:
final_table = pd.DataFrame(table, columns = ["topics"])

In [256]:
final_table

,topics
0,Big Mammals
1,Nature
2,Ecosystem
3,Wildlife
4,People & Exploration
5,Terrain


In [257]:
final_table["Top_Q"] = top_q_list

In [258]:
final_table["Bot_Q"] = bot_q_list

In [259]:
final_table

,topics,Top_Q,Bot_Q
0,Big Mammals,0.163077,0.036034
1,Nature,0.087538,0.352759
2,Ecosystem,0.158308,0.149310
3,Wildlife,0.446615,0.106724
4,People & Exploration,0.044615,0.195172
5,Terrain,0.092923,0.156724
